In [1]:
%cd ../

/home/ruslan/Documents/course-project-TiMP


In [2]:
import pickle
import matplotlib.pyplot as plt
from collections import Counter, OrderedDict
from sortedcontainers import SortedDict
import numpy as np
import seaborn as sns; sns.set()
from  matplotlib import cm

# Создаём гистограмму для колчества функций встречающихся в датасете в зависимости от их частоты.
Огромная функция для построения графика зависимости количества элементов с данной частотой вхождений от этой частоты.

Происходит парсинг вокабов, полученных при препроцессинге, создание гистограммы, а также вычисление размеров отфильтрованных данных для последующего вывода на график.


In [3]:
def historgram(train_vocab, train_name, test_vocab, test_name, val_vocab, val_name, limit):
    plt.figure(figsize=(20, 10))
    train_vocab = {key:int(value) for key, value in train_vocab.items()}
    train = SortedDict(sorted(Counter(train_vocab.values()).items()))
    test_vocab = {key:int(value) for key, value in test_vocab.items()}
    test = SortedDict(sorted(Counter(test_vocab.values()).items()))
    val_vocab = {key:int(value) for key, value in val_vocab.items()}
    val = SortedDict(sorted(Counter(val_vocab.values()).items()))
    combined_keys = {*train.keys(), *test.keys(), *val.keys()}
    train_missing_keys = combined_keys - train.keys()
    train.update({key:0 for key in train_missing_keys})
    test_missing_keys = combined_keys - test.keys()
    test.update({key:0 for key in test_missing_keys})
    val_missing_keys = combined_keys - val.keys()
    val.update({key:0 for key in val_missing_keys})
    
    bad = {key:train[key]+test[key]+val[key] for key in train.keys() if key < limit}
    plt.bar(list(train.keys()), list(train.values()), label=train_name)
    plt.bar(list(test.keys()), list(test.values()), bottom=list(train.values()), color='green', label=test_name)
    plt.bar(list(val.keys()), list(val.values()), bottom=np.sum((list(train.values()), list(test.values())), axis=0), color='red', label=val_name)
    plt.bar(list(bad.keys()), list(bad.values()), color="black", label="filtered", alpha=0.4)
    plt.yscale("log")
    plt.legend()
    plt.xlim(0, 100)
    plt.xlabel("Частота вхождения элемента в AST.")
    plt.ylabel("Количество элементов с такой частотой.")
    bad_quantity = [0,0,0]
    total_quantity = [0, 0, 0]
    for i in range(1, limit):
        bad_quantity[0]+=i*train[i]
        bad_quantity[1]+=i*test[i]
        bad_quantity[2]+=i*val[i]
    for key in train.keys():
        total_quantity[0]+=key*train[key]
        total_quantity[1]+=key*test[key]
        total_quantity[2]+=key*val[key]
    if limit>1:
        plt.title("График для ограничения "+ str(limit))
        plt.suptitle("Для тренировочного датасета: отфильтровано " + str(bad_quantity[0]) + " из " + str(total_quantity[0]) + 
                     ", т. е. "  + "%.2f" % (100 * bad_quantity[0]/total_quantity[0]) + "%.\n"
                     "Для тестового датасета: отфильтровано " + str(bad_quantity[1]) + " из " + str(total_quantity[1]) + 
                     ", т. е. "  + "%.2f" % (100 * bad_quantity[1]/total_quantity[1]) + "%.\n"
                     "Для валидационного датасета: отфильтровано " + str(bad_quantity[2]) + " из " + str(total_quantity[2]) + 
                     ", т. е. "  + "%.2f" % (100 * bad_quantity[2]/total_quantity[2]) + "%.\n"
                    )
    plt.savefig("img.png", dpi=300)

    

Поднимаем данные при помощи pickle, причём первые два словаря в c2v.dict это train_token и train_path, они нам не нужны. После этого загружаем всё в гистограмму.

In [4]:
def dict_info(path):
    with open(path, "rb") as file:
        token = pickle.load(file)
        path = pickle.load(file)
        target = pickle.load(file)
    print("Train", "total:", len(target), "total lines")
    print("Validate", "total:", len(token), "total lines")
    print("Test", "total:", len(path), "total lines")

    historgram(target, "train", path, "test",  token, "validate", int(input()))


In [ ]:
dict_info("dataset/java-small/java-small.c2v.dict")

Train total: 40651 total lines
Validate total: 7441 total lines
Test total: 22878 total lines


# Гистограмма топ N имён функций по встречаемости из всех наборов.

In [ ]:
TRAIN_VOCAB_PATH = "dataset/java-small/java-small.train.functions.vocab"
TEST_VOCAB_PATH = "dataset/java-small/java-small.test.functions.vocab"
VALIDATION_VOCAB_PATH = "dataset/java-small/java-small.validation.functions.vocab"

Загрузим все vocab-ы в pandas-таблицы:

In [ ]:
import pandas as pd

train_df = pd.read_csv(TRAIN_VOCAB_PATH, sep=' ', names=["Function", "Frequency"])
test_df = pd.read_csv(TEST_VOCAB_PATH, sep=' ', names=["Function", "Frequency"])
validation_df = pd.read_csv(VALIDATION_VOCAB_PATH, sep=' ', names=["Function", "Frequency"])

train_df["Dataset"] = ["train"] * len(train_df)
test_df["Dataset"] = ["test"] * len(test_df)
validation_df["Dataset"] = ["validation"] * len(validation_df)

display(train_df.sort_values(by="Function"))
display(test_df.sort_values(by="Function"))
display(validation_df.sort_values(by="Function"))

Сгруппируем по дубликатам и просуммируем:

In [ ]:
conc = pd.concat([train_df, test_df, validation_df], ignore_index=True)
conc = conc.groupby('Function').sum().reset_index()

display(conc)

Построим гистограмму встречаемости слов для заданного датасета


In [ ]:
def target_histogram(data, name, limit = 50):
    cnt = Counter()
    freqs = data["Frequency"]
    targets = data["Function"]
    for seq, freq in zip(targets, freqs):
        cnt.update({seq: freq})

    srt = sorted(cnt.items(), key=lambda x: x[1], reverse=True)
    srt_labels, srt_freqs = list(map(list, zip(*srt)))
    srt_labels = srt_labels[:limit] + ['other words']
    srt_freqs = srt_freqs[:limit] + [sum(srt_freqs[limit:])]
    plt.figure(figsize=(15, 8))
    plt.title(f"Распределение первых {limit} функций по встречаемости в " + name + " датасете")
    plt.yscale("log")
    plt.xticks(rotation='vertical')
    plt.margins(0.025)
    plt.bar(srt_labels, srt_freqs)
    plt.plot();

In [ ]:
target_histogram(conc, "объединённом")
target_histogram(train_df, "тренировочном")
target_histogram(test_df, "тестовом")
target_histogram(validation_df, "валидационном")